## Assignment 5
The following assignment will focus on analysing the differences between the sexes by age in Ireland.
Author: Loic Soares Bagnoud


### Part 1
#### Preparing the Data

In [62]:
import pandas as pd
import numpy as np

In [63]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


Now I can do the code that is the prep for analysis.py  
I just noticed that there were more columns so I need to get their names

In [64]:
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [65]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','C02076V03371','C03789V04537','UNIT']

df.drop(columns=drop_col_list, inplace=True)

df = df[df["Single Year of Age"] != "All ages"]
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')

df = df[df["Sex"] != "Both sexes"]
df.info()

print (df.head(3))


<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      6464 non-null   object
 1   Single Year of Age       6464 non-null   int64 
 2   Administrative Counties  6464 non-null   object
 3   VALUE                    6464 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 252.5+ KB
       Sex  Single Year of Age Administrative Counties  VALUE
3296  Male                   0                 Ireland  29610
3297  Male                   0   Carlow County Council    346
3298  Male                   0     Dublin City Council   3188


In [66]:
df_anal = pd.pivot_table(df, 'VALUE',["Sex",'Single Year of Age'],"Administrative Counties")
print (df_anal.head(3))
# write out the entire file to local machine
df_anal.to_csv("population_for_analysis.csv")

Administrative Counties    Carlow County Council  Cavan County Council  \
Sex    Single Year of Age                                                
Female 0                                   353.0                 501.0   
       1                                   302.0                 477.0   
       2                                   334.0                 520.0   

Administrative Counties    Clare County Council  Cork City Council  \
Sex    Single Year of Age                                            
Female 0                                  691.0             1124.0   
       1                                  704.0             1136.0   
       2                                  744.0             1162.0   

Administrative Counties    Cork County Council  Donegal County Council  \
Sex    Single Year of Age                                                
Female 0                                2055.0                   881.0   
       1                                2045.0          

Reference: 
I had some issues here with an error I was getting. Luckily, ChatGPT helped me understand what the problem was:

 - https://chatgpt.com/share/68fb8980-d6bc-800b-93f3-d0702e0e6ee1

#### Weighted descriptive statistics

In [67]:
df_sex_only = df[['Sex', 'Single Year of Age', 'VALUE']]
df_sex_only

,Sex,Single Year of Age,VALUE
3296,Male,0,29610
3297,Male,0,346
3298,Male,0,3188
3299,Male,0,1269
3300,Male,0,2059
...,...,...,...
9787,Female,100,7
9788,Female,100,9
9789,Female,100,12
9790,Female,100,31


In [77]:
sex_grouped = (df_sex_only.groupby("Sex"))

weighted_mean_result = sex_grouped.apply(lambda g: np.average(g['Single Year of Age'], weights=g['VALUE']))

weighted_mean_by_sex = weighted_mean_result.reset_index(name='Weighted Mean Age')
weighted_mean_by_sex

C:\Users\lourd\AppData\Local\Temp\ipykernel_7868\1955519840.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_mean_result = sex_grouped.apply(lambda g: np.average(g['Single Year of Age'], weights=g['VALUE']))


,Sex,Weighted Mean Age
0,Female,38.939796
1,Male,37.739448


References:

https://www.geeksforgeeks.org/pandas/python-pandas-dataframe-groupby/
https://realpython.com/pandas-reset-index/

#### Calculating the Difference between the sexes by age

#### Or you can use numpy